In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For importing processed data
# processed_data_path = data_path

data = pd.read_csv(processed_data_path, usecols=['datadate', 'TBill1y'])
data = data.drop_duplicates(subset=['datadate'])

In [ ]:
last_simul_day = "2024-12-03"
rfr = list(data[data['datadate'] <= last_simul_day][['datadate', 'TBill1y']].drop_duplicates(subset=['datadate'])['TBill1y'])
rfr = rfr[-67*20:]

all_days = list(data[data['datadate'] <= last_simul_day].datadate.unique())
x_axis = all_days[-len(rfr)-1:]

In [ ]:
# Load files
# file_list = ["file_path"]

ensemble_curves = []

# Calculate Sharpe Ratio
def calculate_SR(total_asset, rfr):
    
    daily_ret = [(total_asset[i] - total_asset[i-1]) / total_asset[i-1] for i in range(1, len(total_asset))]
    assert len(daily_ret) == len(rfr)
    excess_ret = [daily_ret[i] - rfr[i] for i in range(len(rfr))]
    SR = np.mean(excess_ret) / np.std(daily_ret) * np.sqrt(252)
    
    return SR

# Load JSON and extract the net value curves of mixture-of-experts ensembles
for file in file_list:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        for key in data.keys():
            ensemble_curves.append(np.array(data[key]))


# Do equal-weighted average of all mixture-of-experts ensembles
avg_curve = np.zeros(len(ensemble_curves[0]))
avg_curve[0] = 1
for i in range(1, len(avg_curve)):
    avg_curve[i] = avg_curve[i-1]*np.mean([curve[i]/curve[i-1] for curve in ensemble_curves]) 

dates = np.arange(len(avg_curve))

def calculate_cagr(nav):
    '''
    Calculates the annual percentage return
    '''
    years = len(nav) / 252
    return (nav[-1] / nav[0]) ** (1 / years) - 1

def calculate_max_drawdown(nav):
    '''
    Calculates maximum drawdown and duration
    '''
    peak = np.maximum.accumulate(nav)
    drawdown = (nav - peak) / peak
    max_dd = np.min(drawdown)

    # Calculates maximum drawdown duration
    drawdown_end = np.argmax(drawdown)
    drawdown_start = np.where(nav[:drawdown_end] == peak[drawdown_end])[0][0]
    max_dd_duration = drawdown_end - drawdown_start
    return max_dd, max_dd_duration

# Calculates maximum drawdown and duration
cagr = calculate_cagr(avg_curve)
sharpe = calculate_SR(avg_curve, rfr)
print('sharpe',sharpe)
#max_dd, max_dd_duration = calculate_max_drawdown(avg_curve)

plt.figure(figsize=(16, 6))
indices_to_display = np.linspace(0, len(x_axis)-1, 15, dtype=int)
plt.xticks(indices_to_display, [x_axis[i] for i in indices_to_display], rotation=45)
plt.plot(x_axis, avg_curve, label="Ensemble Weighted Avg NAV", color="blue")
plt.ylabel("Net Asset Value (NAV)")
plt.title("Ensemble Weighted Strategy Net Value Curve, 2019-2024")
plt.legend()
plt.grid()
plt.show()